In [1]:
# Nina Prakash
# CMU Mechanical and AI Lab
# Professor Farimani
# June 2019

# Use LSTM to predict 1D non-steady Hagen-Poiseuille flow.
# Assume constant pressure gradient and fluid starts at rest.

In [2]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os

In [3]:
# global variables

# date = '6-27-19'

numPoints = 16 # need this to pre-process
dt = .25 # s
ti = 0 # s
tf = 5 # s    # change back to 50 later
# rho = 888      # of oil at 20 degrees C [kg/m^3]
# mu = 0.8       # of oil at 20 degrees C [kg/m*s]
# nu = mu/rho
# dpdx = -10    # axial pressure gradient [Pa/m = kg/m^2*s^2]
# testSize = 0.2 # set aside 20% for testing

# Ds = np.linspace(0.5,1,num=10000) # increase later (500)

## Load Data

In [4]:
data_dir = 'data/'
# numBCs = 10000
# minD = 0.50
# maxD = 1.00
# numTimeSteps = 20

# data_session_name = '%s_%.2f to %.2f_%s time steps' %(numBCs,minD,maxD,numTimeSteps)
data_session_name = 'all_params_50625'

dataset_filename = os.path.join(data_dir,'dataset_'+data_session_name+'.npy')
bc_filename = os.path.join(data_dir,'bc_'+data_session_name+'.npy')

target = np.load(dataset_filename)
inputs = np.load(bc_filename)

print('dataset loaded of size: ', np.shape(target))



dataset loaded of size:  (50625, 20, 17)


## Pre-Process Data

In [5]:
# replace first time step with all the boundary conditions?
# for now, hardcoded to fill in since there are 4 bc's and 17 points

for i in range(len(inputs)):
    D = inputs[i][0]
    dpdx = inputs[i][1]
    mu = inputs[i][2]
    nu = inputs[i][3]
    for j in range(numPoints+1):
        if 0 <= j < 5:
            target[i][0][j] = D
        elif 5 <= j < 9:
            target[i][0][j] = dpdx
        elif 9 <= j < 13:
            target[i][0][j] = mu
        else:
            target[i][0][j] = nu
        
print('replaced first time step with BCs\n')
# print('shape of inputs: ', np.shape(inputs))
# print('shape of target: ', np.shape(target))
# print('first bc: ', inputs[0])
# print('first target: ', target[0][0])
# print('another bc : ', inputs[7])
# print('another target: ', target[7][0])

replaced first time step with BCs



## LSTM

In [6]:
# adapted from Wei's code, model.py,
# and https://www.jessicayung.com/lstms-for-time-series-in-pytorch/

import torch
import torch.nn as nn
from torch.autograd import Variable

class BasicLSTM(nn.Module):
    
    def __init__(self,hidden_size, num_layers, num_features, device, dropout):
        super(BasicLSTM,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_features = num_features
        self.device = device
        
        if self.num_layers > 1:
            self.dropout = dropout
        else:
            self.dropout = 0.0
        
        # define the LSTM layer
        self.lstm = nn.LSTM(
            input_size = self.num_features,
            hidden_size = self.hidden_size,
            num_layers = self.num_layers,
            batch_first=True,
            dropout=self.dropout
        )
        
        # define the output layer
        self.dense = nn.Linear(self.hidden_size,self.num_features)
        
    # initialize hidden state as
    def initial_hidden_state(self,batch):
        return Variable(torch.zeros(self.num_layers,batch,self.hidden_size).to(self.device))

    # forward pass through LSTM layer
    def forward(self,x):
        batch, _, _ = x.shape
        h_0 = self.initial_hidden_state(batch)
        h_1 = self.initial_hidden_state(batch)
        out, _ = self.lstm(x, (h_0, h_1))
        out = self.dense(out)
        return out

In [7]:
# from Wei's code, utils.py

def expanding_pred(net, truth_seq):
    pred_seq = np.empty(truth_seq.shape)
    pred_seq[0] = truth_seq[0]
    for i in range(1, len(pred_seq)):
        in_ = pred_seq[:i].reshape(1, i, -1)
        if type(in_) is not torch.Tensor:
            in_ = torch.Tensor(in_)
        if in_.dtype is not torch.float32:
            in_ = in_.type(torch.float32)
        out_ = net.predict_proba(in_)
        pred_seq[i] = out_.reshape(i, -1)[-1]
    return pred_seq

In [ ]:
# adapted from Wei's code, main.py

import os
import time
import json
from torch.utils.data import DataLoader, random_split
# import torch.utils.data
# import torch
# print(torch.__version__)
import skorch
import pickle
from sklearn.model_selection import KFold

def main():
    start_time = time.time()
    torch.manual_seed(19)
    torch.cuda.manual_seed(19)
    np.random.seed(19)
    
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    print('device: ', device)
    criterion = nn.MSELoss()
    
    data_dir = 'data/'
    session_name = '1000_epochs_adam_withcrossval'
    result_dir = 'result/' + session_name + '/'
    model_dir = 'model/' + session_name + '/'
    for dir_name in [result_dir, model_dir]:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            
    num_points = 16
    num_features = num_points+1
    num_epochs = 1000 #500
    hidden_size = 196 #32
    num_layers=3
    dropout=0.2
    
    for j in [10000,20000,30000,40000,50000]:
        numBCName = str(j)
    
        print('NOW TRAINING WITH DATASET OF LENGTH ' + numBCName + '\n')
        # data_dir_ = os.path.join(data_dir, numBCName)
        resultDir = os.path.join(result_dir, numBCName)
        modelDir = os.path.join(model_dir,numBCName)
        for dir in  [resultDir, modelDir]:
            if not os.path.exists(dir):
                os.makedirs(dir)
                
        dataset,foo = random_split(target, [j,len(target)-j])
        dataset = np.asarray(dataset)

        # above here is fine. for this size dataset, i need to train a model using k-fold cross val
        ###############
        
        # split dataset

#         test_len = int(0.2 * len(dataset))
#         valid_len = int(0.2 * (len(dataset) - test_len))
#         train_set, valid_set, test_set = random_split(dataset, [len(dataset) - valid_len - test_len, valid_len, test_len])

        kf = KFold(5)
        
        split_num = 0
        
        for train_i, test_i in kf.split(dataset):
            split_num += 1
            train = dataset[train_i]
            train_len = int(0.8*len(train))
            valid_len = int(0.2*len(train))
            train_set, valid_set = random_split(train, [train_len, valid_len])
            test_set = dataset[test_i]
            test_len = len(test_set)
            
            print('Running cross val split number ', split_num)
            print('shape of training set: ', np.shape(train_set))
            print('shape of validation set: ', np.shape(valid_set))
            print('shape of test set: ', np.shape(test_set))
            
            result_dir_ = os.path.join(resultDir, "Split %s" %(split_num))
            model_dir_ = os.path.join(modelDir, "Split %s" %(split_num))
            for dir in  [result_dir_, model_dir_]:
                if not os.path.exists(dir):
                    os.makedirs(dir)

        
            net = skorch.NeuralNetRegressor(
                module=BasicLSTM(hidden_size,num_layers,num_features,device,dropout),
                criterion=nn.MSELoss,
                optimizer=torch.optim.Adam,
                device=device,
                batch_size=-1,
                train_split=None,
                max_epochs=1,
                warm_start=True,
                verbose=0,
            )
            best_net = net
            with open(model_dir + '/model_setting.pkl','wb') as out_model:
                pickle.dump(net, out_model)


            train_history = []
            best_loss = None
            for epoch in range(num_epochs):
                # start_time = time.time()
                history = {'epoch': epoch+1,
                          'best_model': False}
                train_loss, valid_loss = 0, 0

                # train and save average loss
                for i, train_batch in enumerate(DataLoader(train_set,batch_size=128,num_workers=6, pin_memory=True)):
                    X_train, y_train = train_batch[:, :-1].type(torch.float32), train_batch[:,1:].type(torch.float32)
                    net.fit(X_train, y_train)
                    train_loss += net.history[-1]['train_loss']
                train_loss /= i+1

                # validation & save average loss
                for i, train_batch in enumerate(DataLoader(valid_set,batch_size=128,num_workers=6,pin_memory=True)):
                    X_valid, y_valid = train_batch[:, :-1].type(torch.float32), train_batch[:,1:].type(torch.float32)
                    y_pred_valid = torch.Tensor(net.predict_proba(X_valid))
                    valid_loss += criterion(y_pred_valid, y_valid).detach().numpy()
                valid_loss /= i+1
                history['train_loss'] = train_loss
                history['valid_loss'] = valid_loss
                history['duration'] = time.time() - start_time

                # if valid loss best: save the model
                if best_loss is None or best_loss > valid_loss:
                    best_loss = valid_loss
                    history['best_model'] = True
                    net.save_params(f_params=model_dir_+'/model.pt', f_optimizer=model_dir_+'/optimizer.pt')
                train_history.append(history)
                print(train_history[-1])

            # save training history
            for last_epoch in train_history[::-1]:
                if last_epoch['best_model']:
                    print('Best model is achieved after {} epochs, validation loss = {:.4f}'
                          .format(last_epoch['epoch'], last_epoch['valid_loss']))
                    break
            with open(model_dir_+'/history.json', 'w') as output:
                json.dump(train_history, output)

         # initialize the best model and predict
            best_net.initialize()
            best_net.load_params(
                f_params=model_dir_+'/model.pt', f_optimizer=model_dir_+'/optimizer.pt'
            )

            for _, test_data in enumerate(DataLoader(test_set, batch_size=test_len)):
                y_pred = np.empty(test_data.numpy().shape)
                print('Testing')
                for i in range(len(y_pred)):
                    y_pred[i] = expanding_pred(best_net, test_data[i])
                    print('Testing progress: {}/{}'.format(i + 1, len(y_pred)), end='\r')
                print()

                if not os.path.exists(result_dir_):
                    os.makedirs(result_dir_)
                np.save(os.path.join(result_dir_, 'y_pred.npy'), np.float32(y_pred))
                np.save(os.path.join(result_dir_, 'y_test.npy'), test_data.numpy())
    
    elapsed_time = time.time() - start_time()
    print('time elapsed: ', elapsed_time)


if __name__ == '__main__':
    main()

device:  cuda:0
NOW TRAINING WITH DATASET OF LENGTH 10000

Running cross val split number  1
shape of training set:  (6400, 20, 17)
shape of validation set:  (1600, 20, 17)
shape of test set:  (2000, 20, 17)
{'epoch': 1, 'best_model': True, 'train_loss': 0.0009260744679340859, 'valid_loss': 1.9546041305427654e-05, 'duration': 2.3596911430358887}
{'epoch': 2, 'best_model': True, 'train_loss': 1.874642726761522e-05, 'valid_loss': 8.906474179709152e-06, 'duration': 3.3043100833892822}
{'epoch': 3, 'best_model': True, 'train_loss': 1.0934262245427818e-05, 'valid_loss': 6.108414289393784e-06, 'duration': 4.205512285232544}
{'epoch': 4, 'best_model': True, 'train_loss': 8.01314025920874e-06, 'valid_loss': 4.8613687795295846e-06, 'duration': 5.198734760284424}
{'epoch': 5, 'best_model': True, 'train_loss': 6.526905281134532e-06, 'valid_loss': 4.477385398506438e-06, 'duration': 6.1215736865997314}
{'epoch': 6, 'best_model': True, 'train_loss': 5.380805359891383e-06, 'valid_loss': 3.50698209891

{'epoch': 58, 'best_model': True, 'train_loss': 4.6734365525935574e-07, 'valid_loss': 1.4431048410957513e-07, 'duration': 93.30668449401855}
{'epoch': 59, 'best_model': True, 'train_loss': 4.826862704021551e-07, 'valid_loss': 1.3165278890990243e-07, 'duration': 95.72223567962646}
{'epoch': 60, 'best_model': False, 'train_loss': 6.661579402589268e-07, 'valid_loss': 4.581479144095423e-07, 'duration': 98.19553399085999}
{'epoch': 61, 'best_model': False, 'train_loss': 7.943784208919169e-07, 'valid_loss': 1.3858876505205094e-06, 'duration': 100.61348533630371}
{'epoch': 62, 'best_model': False, 'train_loss': 5.68012175676813e-07, 'valid_loss': 1.5748294948445198e-07, 'duration': 103.13963794708252}
{'epoch': 63, 'best_model': False, 'train_loss': 2.6222539560194493e-06, 'valid_loss': 3.855840282959924e-07, 'duration': 105.66471338272095}
{'epoch': 64, 'best_model': False, 'train_loss': 4.7526513412776695e-07, 'valid_loss': 1.6416675062093697e-07, 'duration': 108.1624162197113}
{'epoch': 65

{'epoch': 116, 'best_model': False, 'train_loss': 4.0682692286964084e-07, 'valid_loss': 8.907370239687505e-08, 'duration': 274.4333703517914}
{'epoch': 117, 'best_model': False, 'train_loss': 9.712277341122898e-07, 'valid_loss': 1.9596306690171053e-07, 'duration': 278.28715109825134}
{'epoch': 118, 'best_model': False, 'train_loss': 4.545703001213042e-07, 'valid_loss': 5.248284126607289e-07, 'duration': 282.199419260025}
{'epoch': 119, 'best_model': False, 'train_loss': 3.996571473408039e-07, 'valid_loss': 1.505106308303976e-07, 'duration': 286.19034242630005}
{'epoch': 120, 'best_model': False, 'train_loss': 1.3485056547324347e-06, 'valid_loss': 5.107383655146035e-07, 'duration': 290.3359546661377}
{'epoch': 121, 'best_model': False, 'train_loss': 3.910473799351166e-07, 'valid_loss': 1.5564226338518404e-07, 'duration': 294.60775566101074}
{'epoch': 122, 'best_model': False, 'train_loss': 5.105715024456003e-07, 'valid_loss': 2.4849883135839963e-07, 'duration': 298.77170753479004}
{'epo

{'epoch': 174, 'best_model': False, 'train_loss': 4.148567381889734e-07, 'valid_loss': 1.2407048079519603e-07, 'duration': 551.1782767772675}
{'epoch': 175, 'best_model': False, 'train_loss': 4.0840292911070717e-07, 'valid_loss': 1.3212174164979072e-07, 'duration': 556.774703502655}
{'epoch': 176, 'best_model': False, 'train_loss': 4.786083133012653e-07, 'valid_loss': 1.5799642264587769e-07, 'duration': 562.2503333091736}
{'epoch': 177, 'best_model': False, 'train_loss': 3.6401597697022226e-07, 'valid_loss': 1.21652974463324e-07, 'duration': 567.7940125465393}
{'epoch': 178, 'best_model': False, 'train_loss': 3.8965090766396314e-07, 'valid_loss': 3.376485962619951e-07, 'duration': 573.4522082805634}
{'epoch': 179, 'best_model': False, 'train_loss': 4.221591581199391e-07, 'valid_loss': 2.1452549740038543e-07, 'duration': 579.0675532817841}
{'epoch': 180, 'best_model': False, 'train_loss': 5.46019392118069e-07, 'valid_loss': 2.8486493000365533e-07, 'duration': 584.889237165451}
{'epoch':

{'epoch': 232, 'best_model': False, 'train_loss': 2.821767617433579e-07, 'valid_loss': 4.73537474258242e-07, 'duration': 921.6829459667206}
{'epoch': 233, 'best_model': False, 'train_loss': 2.5268140404932635e-07, 'valid_loss': 2.867406491392243e-07, 'duration': 928.9530997276306}
{'epoch': 234, 'best_model': False, 'train_loss': 2.850891539196709e-07, 'valid_loss': 3.2710904590133464e-07, 'duration': 936.4342048168182}
{'epoch': 235, 'best_model': False, 'train_loss': 3.807080872775259e-07, 'valid_loss': 1.1083579100666331e-07, 'duration': 943.8725304603577}
{'epoch': 236, 'best_model': False, 'train_loss': 6.027198261904232e-07, 'valid_loss': 9.276492545058528e-08, 'duration': 951.3256013393402}
{'epoch': 237, 'best_model': True, 'train_loss': 2.0766801156923975e-07, 'valid_loss': 4.263998330491002e-08, 'duration': 958.876439332962}
{'epoch': 238, 'best_model': False, 'train_loss': 3.7704790599946137e-07, 'valid_loss': 3.4839113665769287e-07, 'duration': 966.4671497344971}
{'epoch': 

...

{'epoch': 240, 'best_model': False, 'train_loss': 8.429508966401045e-07, 'valid_loss': 1.488695508854969e-07, 'duration': 753689.3477575779}
{'epoch': 241, 'best_model': False, 'train_loss': 3.7756350754610445e-07, 'valid_loss': 3.0265062521625383e-07, 'duration': 753760.4173858166}
{'epoch': 242, 'best_model': False, 'train_loss': 2.7113154685783532e-06, 'valid_loss': 4.933976580751301e-07, 'duration': 753830.7000491619}
{'epoch': 243, 'best_model': False, 'train_loss': 3.8170295871016907e-07, 'valid_loss': 1.31370630116946e-07, 'duration': 753901.3823604584}
{'epoch': 244, 'best_model': False, 'train_loss': 2.816525728614276e-07, 'valid_loss': 1.403174947929648e-07, 'duration': 753972.2297484875}
{'epoch': 245, 'best_model': False, 'train_loss': 2.9431798784192625e-07, 'valid_loss': 1.3831452979442666e-07, 'duration': 754043.5112085342}
{'epoch': 246, 'best_model': False, 'train_loss': 3.843366345298212e-07, 'valid_loss': 1.357544129040147e-07, 'duration': 754115.2643098831}
{'epoch'

{'epoch': 298, 'best_model': False, 'train_loss': 1.1472920922036186e-06, 'valid_loss': 1.98063203826175e-07, 'duration': 758316.8124899864}
{'epoch': 299, 'best_model': False, 'train_loss': 3.5738098659976457e-07, 'valid_loss': 1.1648875174405236e-07, 'duration': 758405.8394601345}
{'epoch': 300, 'best_model': False, 'train_loss': 3.532886508613349e-07, 'valid_loss': 7.132991599016685e-07, 'duration': 758494.6735098362}
{'epoch': 301, 'best_model': False, 'train_loss': 5.653570638249524e-07, 'valid_loss': 2.411279353964346e-07, 'duration': 758584.8039498329}
{'epoch': 302, 'best_model': False, 'train_loss': 5.479590451310893e-07, 'valid_loss': 5.912083203814913e-07, 'duration': 758674.8881192207}
{'epoch': 303, 'best_model': False, 'train_loss': 5.904441054364421e-07, 'valid_loss': 8.295853574150226e-07, 'duration': 758765.5946502686}
{'epoch': 304, 'best_model': False, 'train_loss': 4.4936833982471095e-07, 'valid_loss': 2.8625377186280706e-06, 'duration': 758856.3052194118}
{'epoch':

{'epoch': 356, 'best_model': False, 'train_loss': 4.15266980648236e-07, 'valid_loss': 1.044814466336232e-06, 'duration': 764068.9499878883}
{'epoch': 357, 'best_model': False, 'train_loss': 4.2122534864574846e-07, 'valid_loss': 1.4112839219131656e-06, 'duration': 764177.8558065891}
{'epoch': 358, 'best_model': False, 'train_loss': 5.385069225856872e-07, 'valid_loss': 2.9842499166558186e-07, 'duration': 764287.9786128998}
{'epoch': 359, 'best_model': False, 'train_loss': 4.695808462429341e-07, 'valid_loss': 1.3558950715414062e-07, 'duration': 764398.1704902649}
{'epoch': 360, 'best_model': False, 'train_loss': 8.466903345076086e-07, 'valid_loss': 1.517294113508988e-07, 'duration': 764508.029001236}
{'epoch': 361, 'best_model': False, 'train_loss': 3.449265807375923e-07, 'valid_loss': 1.1543335139214449e-07, 'duration': 764618.5038950443}
{'epoch': 362, 'best_model': False, 'train_loss': 4.0897731480527e-07, 'valid_loss': 3.8124180808776335e-07, 'duration': 764729.3609523773}
{'epoch': 3

{'epoch': 414, 'best_model': False, 'train_loss': 6.183571885761315e-07, 'valid_loss': 2.502075754537863e-07, 'duration': 770980.3739113808}
{'epoch': 415, 'best_model': False, 'train_loss': 2.9190262637257545e-07, 'valid_loss': 2.0942930683232638e-07, 'duration': 771107.8658394814}
{'epoch': 416, 'best_model': False, 'train_loss': 3.1582941829810805e-07, 'valid_loss': 5.025568209535586e-07, 'duration': 771236.9430029392}
{'epoch': 417, 'best_model': False, 'train_loss': 3.380296774935232e-07, 'valid_loss': 5.327449680467716e-07, 'duration': 771366.0853025913}
{'epoch': 418, 'best_model': False, 'train_loss': 1.472043927700876e-06, 'valid_loss': 1.343533669814621e-07, 'duration': 771496.1221208572}
{'epoch': 419, 'best_model': False, 'train_loss': 2.565432429454025e-07, 'valid_loss': 1.7492873526311756e-07, 'duration': 771625.7999162674}
{'epoch': 420, 'best_model': False, 'train_loss': 2.642974490640881e-07, 'valid_loss': 1.324683655399751e-07, 'duration': 771755.7531988621}
{'epoch':

{'epoch': 472, 'best_model': False, 'train_loss': 1.8433311292180103e-06, 'valid_loss': 2.893547775185748e-07, 'duration': 779050.449950695}
{'epoch': 473, 'best_model': False, 'train_loss': 4.4499885310263683e-07, 'valid_loss': 2.63404205207663e-07, 'duration': 779200.0716862679}
{'epoch': 474, 'best_model': False, 'train_loss': 4.854327520812755e-07, 'valid_loss': 2.944423865427014e-07, 'duration': 779350.1811981201}
{'epoch': 475, 'best_model': False, 'train_loss': 4.683213453896921e-07, 'valid_loss': 1.9746360411126693e-07, 'duration': 779500.8726902008}
{'epoch': 476, 'best_model': False, 'train_loss': 4.793635639543936e-07, 'valid_loss': 6.644827019379974e-07, 'duration': 779652.2463612556}
{'epoch': 477, 'best_model': False, 'train_loss': 4.5472141370813307e-07, 'valid_loss': 3.8201064449750493e-07, 'duration': 779803.4483940601}
{'epoch': 478, 'best_model': False, 'train_loss': 4.7288981988913295e-07, 'valid_loss': 2.5126006949892e-07, 'duration': 779954.6423339844}
{'epoch': 4

## Plot Model

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

# date='6-27-19'

session_name = '1000_epochs_adam'
result_dir = 'result/' + session_name + '/'

for k in range(10000,60000,10000):
    numBCName = str(k)
    result_dir_ = os.path.join(result_dir, numBCName)
    filename_pred = os.path.join(result_dir_, 'y_pred.npy')
    filename_test = os.path.join(result_dir_, 'y_test.npy')
    y_pred = np.load(filename_pred)
    y_test = np.load(filename_test)
    print(np.shape(y_pred))
    print(np.shape(y_test))

    test_i = 1500

    numPoints = 16

    D = y_pred[test_i][0][0]
    r = []
    interval = D/numPoints
    x = int(numPoints/2)
    for j in range(-x,x+1):
        r.append(interval*j)
    
    figNum = int(k/10000)
    plt.figure(num=figNum,dpi=600)
    for i in range(len(y_test[test_i])):
        if i != 0:
            plt.plot(y_test[test_i][i],r,label='t = %s' %(i))
    plt.title('True Velocity Profile Development for D = %fm' %(D))
    plt.xlabel('velocity [m/s]')
    plt.ylabel('r [m]')
    plt.legend(bbox_to_anchor=(1,1),fontsize='x-small')

    plt.figure(num=figNum+5,dpi=600)
    for i in range(len(y_pred[test_i])):
        if i != 0:
            plt.plot(y_pred[test_i][i],r,label='t = %s' %(i))
    plt.title('Predicted Velocity Profile Development for D = %fm' %(D))
    plt.xlabel('velocity [m/s]')
    plt.ylabel('r [m]')
    plt.legend(bbox_to_anchor=(1,1),fontsize='x-small')


## Performance Metrics

In [ ]:
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

numBCs = 10000
minD = 0.50
maxD = 1.00
numTimeSteps = 20

session_name = '1000_epochs_adam'
result_dir = 'result/' + session_name + '/'

for i in range(10000,60000,10000):
    numBCName = str(i)
    result_dir_ = os.path.join(result_dir, numBCName)
    filename_pred = os.path.join(result_dir_, 'y_pred.npy')
    filename_test = os.path.join(result_dir_, 'y_test.npy')
    y_pred = np.load(filename_pred)
    y_test = np.load(filename_test)
    filename_text = os.path.join(result_dir_, 'performance metrics ' + numBCName + ' data points')

    num_samples = len(y_pred)

    y_pred = y_pred.reshape(num_samples * numTimeSteps, -1)
    y_test = y_test.reshape(y_pred.shape)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    mse_text = 'mse: %.10f' %(mse)
    mae_text = 'mae: %.10f' %(mae)

    file = open(filename_text, 'w')
    file.write("Performance Metrics \n")
    file.write(mse_text +'\n')
    file.write(mae_text)
    file.close()

